## 2101. Detonate the Maximum Bombs
- Description:
  <blockquote>
    You are given a list of bombs. The **range** of a bomb is defined as the area where its effect can be felt. This area is in the shape of a **circle** with the center as the location of the bomb.
   
  The bombs are represented by a **0-indexed** 2D integer array `bombs` where `bombs[i] = [xi, yi, ri]`. `xi` and `yi` denote the X-coordinate and Y-coordinate of the location of the `ith` bomb, whereas `ri` denotes the **radius** of its range.
   
  You may choose to detonate a **single** bomb. When a bomb is detonated, it will detonate **all bombs** that lie in its range. These bombs will further detonate the bombs that lie in their ranges.
   
  Given the list of `bombs`, return  *the **maximum** number of bombs that can be detonated if you are allowed to detonate **only one** bomb* .
   
  **Example 1:**
  ![Image](https://assets.leetcode.com/uploads/2021/11/06/desmos-eg-3.png)
   
  **Input:** bombs = `[2, 1,3],[6,1,4]`
  **Output:** 2
  **Explanation:**
  The above figure shows the positions and ranges of the 2 bombs.
  If we detonate the left bomb, the right bomb will not be affected.
  But if we detonate the right bomb, both bombs will be detonated.
  So the maximum bombs that can be detonated is max(1, 2) = 2.
   
  **Example 2:**
  ![Image](https://assets.leetcode.com/uploads/2021/11/06/desmos-eg-2.png)
   
  **Input:** bombs = `[1, 1,5],[10,10,5]`
  **Output:** 1Explanation:Detonating either bomb will not detonate the other bomb, so the maximum number of bombs that can be detonated is 1.
   
  **Example 3:**
  ![Image](https://assets.leetcode.com/uploads/2021/11/07/desmos-eg1.png)
   
  **Input:** bombs = `[1, 2,3],[2,3,1],[3,4,2],[4,5,3],[5,6,4]`
  **Output:** 5
  **Explanation:**
  The best bomb to detonate is bomb 0 because:
  - Bomb 0 detonates bombs 1 and 2. The red circle denotes the range of bomb 0.
  - Bomb 2 detonates bomb 3. The blue circle denotes the range of bomb 2.
  - Bomb 3 detonates bomb 4. The green circle denotes the range of bomb 3.
  Thus all 5 bombs are detonated.
   
  **Constraints:**
   
  - `1 <= bombs.length <= 100`
  - `bombs[i].length == 3`
  - `1 <= xi, yi, ri<= 105`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/detonate-the-maximum-bombs/description)

- Topics: DFS, BFS, Graph. Math

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, DFS Recursive
Solution description
- Time Complexity: O(N^3)
  - Building the graph takes O(n^2) time.
  - The time complexity of a typical DFS is O(V+E), there are n nodes and n^2 edges in this problem.
  - Each node is only visited once, which takes O(n) time.
  - For each node, we may need to explore up to n−1 edges to find all its neighbors. Since there are n nodes, the total number of edges we explore is at most n(n−1)=O(n^2).
  - We need to perform n depth-first searches.
  - Total time: n * O(n²) = O(n³)
- Space Complexity: O(N^2)
  - The graph (graph) stores up to O(n²) edges (each node can connect to all others).
  - The visited set during DFS uses O(n) space (stack depth).
  - The call stack of dfs contains also takes n space.
  - Total space: O(n²) (dominated by the graph).

In [ ]:
from collections import defaultdict


class Solution:
    def maximumDetonation(self, bombs: List[List[int]]) -> int:
        graph = defaultdict(list)
        n = len(bombs)
        
        # Build the graph
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue         
                xi, yi, ri = bombs[i]
                xj, yj, _ = bombs[j]

                # Create a path from node i to node j, if bomb i detonates bomb j.
                if ri ** 2 >= (xi - xj) ** 2 + (yi - yj) ** 2:
                    graph[i].append(j)

        # DFS to get the number of nodes reachable from a given node cur
        def dfs(cur, visited):
            visited.add(cur)
            for neib in graph[cur]:
                if neib not in visited:
                    dfs(neib, visited)
            return len(visited)
        
        answer = 0
        # Loop through every bomb index and for each bomb dfs wil return all the other bombs it can detonate, 
        # AKA the longest chain of connected bombs if we start from this bomb index
        for i in range(n):
            visited = set()
            answer = max(answer, dfs(i, visited))
        
        return answer

In [ ]:
# Slightly different way to write dfs method

class Solution:
    def maximumDetonation(self, bombs: List[List[int]]) -> int:
        n = len(bombs)
        explosionRange = defaultdict(list)
        longest_chain = 0

        def inRange(bomb1, bomb2):
            x1, y1, r1 = bomb1[0], bomb1[1], bomb1[2]
            x2, y2 = bomb2[0], bomb2[1]

            return (x1-x2)**2+(y1-y2)**2 <= r1**2
        
        # Build the graph
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                else:
                    if inRange(bombs[i], bombs[j]):
                        explosionRange[i].append(j)
        
        # print(f"explosionRange = {explosionRange}")
        
        # DFS to get the number of nodes that can be visisted from a given node
        def dfs(idx, visited = None):
            if visited is None:
                visited = set()
            
            visited.add(idx)

            for adj_bomb in explosionRange[idx]:
                if adj_bomb not in visited:
                    dfs(adj_bomb, visited)
            
            return len(visited)
        
        for i in range(n):
            longest_chain = max(longest_chain, dfs(i))

        return longest_chain

### Variation of this question where we return the bomb (x,y,r) which causes the max number of other bombs to deteonate, aka return the most destructive bomb/mine

In [ ]:
class Solution:
    def maximumDetonation(self, bombs: List[List[int]]) -> int:
        n = len(bombs)
        explosionRange = defaultdict(list)
        longest_chain = 0

        def inRange(bomb1, bomb2):
            x1, y1, r1 = bomb1[0], bomb1[1], bomb1[2]
            x2, y2 = bomb2[0], bomb2[1]

            return (x1-x2)**2+(y1-y2)**2 <= r1**2
        
        # Build the graph
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                else:
                    if inRange(bombs[i], bombs[j]):
                        explosionRange[i].append(j)
        
        # print(f"explosionRange = {explosionRange}")
        
        # DFS to get the number of nodes that can be visisted from a given node
        def dfs(idx, visited = None):
            if visited is None:
                visited = set()
            
            visited.add(idx)

            for adj_bomb in explosionRange[idx]:
                if adj_bomb not in visited:
                    dfs(adj_bomb, visited)
            
            return len(visited)
        
        # Changed Code
        max_bomb_idx = -1
        for i in range(n):
            cur_chain = dfs(i)
            if cur_chain > longest_chain:
                longest_chain = cur_chain
                max_bomb_idx = i

        return bombs[max_bomb_idx]

##### Returning All "Most Destructive" Bombs
If your goal is to identify every bomb that can trigger the maximum chain reaction, you can collect their indices into a list.

```python
max_detonations = 0
most_destructive_indices = []

for i in range(n):
    visited = set()
    current_count = dfs(i, visited)
    
    if current_count > max_detonations:
        max_detonations = current_count
        # Found a new record! Clear the list and add this index
        most_destructive_indices = [i]
    elif current_count == max_detonations:
        # Tied with the current record, add to the list
        most_destructive_indices.append(i)

# Returns a list of all indices that reach the max count
return most_destructive_indices
```

#### Using a Tie-Breaker (e.g., Largest Radius)
In a real-world scenario (or a more complex coding challenge), you might want to break a tie by choosing the "strongest" bomb—perhaps the one with the largest radius among those that tied for the max count.

```python
max_detonations = 0
best_bomb_idx = -1

for i in range(n):
    visited = set()
    count = dfs(i, visited)
    
    # Update if we find a higher count OR if the count is tied 
    # but the current bomb has a larger radius than our previous best.
    if count > max_detonations:
        max_detonations = count
        best_bomb_idx = i
    elif count == max_detonations:
        if bombs[i][2] > bombs[best_bomb_idx][2]: # radius comparison
            best_bomb_idx = i

return best_bomb_idx
```

### Solution 2, DFS Iterative
Solution description
- Time Complexity: O(N^3)
  - Building the graph takes O(n^2) time.
  - The time complexity of a typical DFS is O(V+E), there are n nodes and n^2 edges in this problem.
  - Each node is only visited once, which takes O(n) time.
  - For each node, we may need to explore up to n−1 edges to find all its neighbors. Since there are n nodes, the total number of edges we explore is at most n(n−1)=O(n^2).
  - We need to perform n depth-first searches.
  - Total time: n * O(n²) = O(n³)
- Space Complexity: O(N^2)
  - The graph (graph) stores up to O(n²) edges (each node can connect to all others).
  - The visited set during DFS uses O(n) space
  - We use a stack stack to store all the nodes to be visited, and in the worst-case scenario, there may be O(n) nodes in stack.
  - Total space: O(n²) (dominated by the graph).

In [ ]:
class Solution:
    def maximumDetonation(self, bombs: List[List[int]]) -> int:
        graph = defaultdict(list)
        n = len(bombs)
        
        # Build the graph
        for i in range(n):
            for j in range(n):
                xi, yi, ri = bombs[i]
                xj, yj, _ = bombs[j]
                
                # Create a path from i to j, if bomb i detonates bomb j.
                if ri ** 2 >= (xi - xj) ** 2 + (yi - yj) ** 2:
                    graph[i].append(j)
        
        def dfs(i):
            stack = [i]
            visited = set([i])
            while stack:
                cur = stack.pop()
                for neib in graph[cur]:
                    if neib not in visited:
                        visited.add(neib)
                        stack.append(neib)
            return len(visited)
        
        answer = 0
        for i in range(n):
            answer = max(answer, dfs(i))
        
        return answer

### Solution 3, BFS
Solution description
- Time Complexity: O(N^3)
  - Building the graph takes O(n^2) time.
  - The time complexity of a typical BFS is O(V+E), there are n nodes and at most n^2 edges in this problem.
  - Each node is enqueued and dequeued once, it takes O(n) to handle all nodes.
  - For each node, we may need to explore up to n−1 edges to find all its neighbors. Since there are n nodes, the total number of edges we explore is at most n(n−1)=O(n^2).
  - We need to perform n breadth-first searches.
  - Total time: n * O(n²) = O(n³)
- Space Complexity: O(N^2)
  - The graph (graph) stores up to O(n²) edges (each node can connect to all others).
  - queue can store up to n nodes.

In [ ]:
from collections import deque


class Solution:
    def maximumDetonation(self, bombs: List[List[int]]) -> int:
        graph = defaultdict(list)
        n = len(bombs)
        
        # Build the graph
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                xi, yi, ri = bombs[i]
                xj, yj, _ = bombs[j]
                
                # Create a path from node i to node j, if bomb i detonates bomb j.
                if ri ** 2 >= (xi - xj) ** 2 + (yi - yj) ** 2:
                    graph[i].append(j)
        
        def bfs(i):
            queue = deque([i])
            visited = set([i])
            while queue:
                cur = queue.popleft()
                for neib in graph[cur]:
                    if neib not in visited:
                        visited.add(neib)
                        queue.append(neib)
            return len(visited)
        
        answer = 0
        for i in range(n):
            answer = max(answer, bfs(i))
        
        return answer